In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import yaml
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from time import sleep
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
halvings = ['2012-11-28', '2016-07-09', '2020-05-11', '2024-05-04']

In [ ]:
config_file = 'config.yml'
with open(config_file, 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

# insert your API key here
API_KEY = cfg['api_glassnode']['api_glassnode_key']

In [ ]:
# price usd
res = requests.get('https://api.glassnode.com/v1/metrics/market/price_usd_close',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_price = pd.read_json(res.text, convert_dates=['t'])
df_price.set_index('t', inplace=True)
df_price.rename(columns = {'v':'ClosePrice'}, inplace = True)

In [ ]:
# ohlc usd
# i: interval (1h | 24h)
res = requests.get('https://api.glassnode.com/v1/metrics/market/price_usd_ohlc',
    params={'a': 'BTC', 'i': '24h', 'api_key': API_KEY})

# convert to pandas dataframe
df_price_ohlc = pd.read_json(res.text, convert_dates=['t'])
df_price_ohlc['OpenPrice'] = [d.get('o') for d in df_price_ohlc['o']]
df_price_ohlc['HighPrice'] = [d.get('h') for d in df_price_ohlc['o']]
df_price_ohlc['LowPrice'] = [d.get('l') for d in df_price_ohlc['o']]
df_price_ohlc['ClosePrice'] = [d.get('c') for d in df_price_ohlc['o']]

df_price_ohlc.drop('o', inplace=True, axis=1)
df_price_ohlc.set_index('t', inplace=True)

In [ ]:
df_price_ohlc[df_price_ohlc['ClosePrice'].isna()]

In [ ]:
# Hash Rate
res = requests.get('https://api.glassnode.com/v1/metrics/mining/hash_rate_mean',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
list_hash_rate = list(eval(res.text))
t = [d['t'] for d in list_hash_rate]
v = [d['v'] for d in list_hash_rate]
df_hash_rate = pd.DataFrame(columns=[
        't', 
        'v'])

df_hash_rate['t'] = t
df_hash_rate['v'] = v
df_hash_rate['t'] = pd.to_datetime(df_hash_rate['t'], unit='s')
df_hash_rate['v'] = df_hash_rate['v'].astype('float')

df_hash_rate.set_index('t', inplace=True)
df_hash_rate = pd.merge(df_hash_rate, df_price, how='left', on='t')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_hash_rate.index, y=df_hash_rate['ClosePrice'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=df_hash_rate.index, y=df_hash_rate['v'], name='Hash Rate', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)

fig.update_layout(title='Mean Hash Rate',
                  width=1400, height=600)

fig.show()

In [ ]:
# SOPR
res = requests.get('https://api.glassnode.com/v1/metrics/indicators/sopr',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_sopr = pd.read_json(res.text, convert_dates=['t'])
df_sopr.set_index('t', inplace=True)

df_sopr = pd.merge(df_sopr, df_price, how='left', on='t')
df_sopr = df_sopr['2012-12-31':]

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_sopr.index, y=df_sopr['ClosePrice'], name="Price"),
    secondary_y=True)

fig.add_trace(
    go.Scatter(x=df_sopr.index, y=df_sopr['v'], name="SOPR"),
    secondary_y=False)
fig.add_hline(y=1, line_color="black", secondary_y=False)

# Add figure title
fig.update_layout(
    title_text='Spent Output Profit Ratio (SOPR)',
    hovermode='x unified',
    legend=dict(orientation="h"))

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='SOPR', secondary_y=False)

fig.show()

In [ ]:
# Active addresses
res = requests.get('https://api.glassnode.com/v1/metrics/addresses/active_count',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_active = pd.read_json(res.text, convert_dates=['t'])
df_active.set_index('t', inplace=True)
df_active = pd.merge(df_active, df_price, how='left', on='t')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_active.index, y=df_active['ClosePrice'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=df_active.index, y=df_active['v'], name='Active Addresses', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)

fig.update_layout(title='Number of Active Addresses',
                  width=1400, height=600)

fig.show()

In [ ]:
# transactions count
res = requests.get('https://api.glassnode.com/v1/metrics/transactions/count',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_transactions = pd.read_json(res.text, convert_dates=['t'])
df_transactions.set_index('t', inplace=True)
df_transactions = pd.merge(df_transactions, df_price, how='left', on='t')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_transactions.index, y=df_transactions['ClosePrice'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=df_transactions.index, y=df_transactions['v'], name='Transactions', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)

fig.update_layout(title='Number of Transactions',
                  width=1400, height=600)

fig.show()

In [ ]:
#mining/difficulty_latest
res = requests.get('https://api.glassnode.com/v1/metrics/mining/difficulty_latest',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
list_difficulty = list(eval(res.text))
t = [d['t'] for d in list_difficulty]
v = [d['v'] for d in list_difficulty]
df_difficulty = pd.DataFrame(columns=[
        't', 
        'v'])

df_difficulty['t'] = t
df_difficulty['v'] = v
df_difficulty['t'] = pd.to_datetime(df_difficulty['t'], unit='s')
df_difficulty['v'] = df_difficulty['v'].astype('float')
df_difficulty.set_index('t', inplace=True)
df_difficulty = pd.merge(df_difficulty, df_price, how='left', on='t')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_difficulty.index, y=df_difficulty['ClosePrice'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=df_difficulty.index, y=df_difficulty['v'], name='Difficulty', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)

fig.update_layout(title='Mining Difficulty',
                  width=1400, height=600)

fig.show()

## Stock-to-Flow model (S2F)

The stock-to-flow model uses a simple formula where you divide the current supply (stock) of an asset by its yearly production (flow). 

S2F = Stock (existing stock pile) / Flow (annual production)

When you divide Bitcoin’s current circulating supply by the annual rate of new coins entering the supply, you get the Bitcoin stock-to-flow rate. 

At the time of writing this article, the total Bitcoin supply is 18,952,362 while the annual production is 328,500BTC. The annual production is calculated by multiplying the 900 BTC mined per day by 365 days. About 900 BTC are mined daily, which is calculated by multiplying the number of blocks mined per day (144) by the current block subsidy of 6.25 BTC.

In [ ]:
# stock_to_flow_ratio
# https://medium.com/@100trillionUSD/modeling-bitcoins-value-with-scarcity-91fa0fc03e25
from technical_indicator_utils import get_rsi

res = requests.get('https://api.glassnode.com/v1/metrics/indicators/stock_to_flow_ratio',
    params={'a': 'BTC', 'api_key': API_KEY})

# convert to pandas dataframe
df_s2f = pd.read_json(res.text, convert_dates=['t'])
df_s2f['daysTillHalving'] = [d.get('daysTillHalving') for d in df_s2f['o']]
df_s2f['ratio'] = [d.get('ratio') for d in df_s2f['o']]
df_s2f.drop('o', inplace=True, axis=1)
df_s2f.set_index('t', inplace=True)

result = pd.merge(df_s2f, df_price, how='left', on='t')
result['error'] = np.log(result['ClosePrice']) - np.log(result['ratio'])
result['error'] = result['error']/result['error'].abs().max()
result['RSI'] = get_rsi(result['ClosePrice'])
#result = result['2011-04-01':]

halvings = ['2012-11-28', '2016-07-09', '2020-05-11', '2024-05-04']

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.01, 
                    row_heights=[0.8,0.2])
fig.add_trace(go.Scatter(x=result.index, 
                         y=result['ClosePrice'],
                         name='Price',
                         mode='markers',
                         marker=dict(
                            size=6,
                            color=result['daysTillHalving'], #set color equal to a variable
                            colorscale='Rainbow', # one of plotly colorscales
                            colorbar=dict(
                                title="Days until next halving",
                                titleside="right"
                            ),
                            showscale=True)))
fig.add_trace(go.Scatter(x=result.index, 
                         y=result['ratio'],
                         name='Ratio',
                         mode='lines'))
for d in halvings:
    fig.add_vline(x=d, line_dash='dash', line_color='gray')

# Row 2
fig.add_trace(go.Scatter(x=result.index, 
                         y=result['error'],
                         name='Variance',
                         mode='lines'), row=2, col=1)
fig.add_hline(y=0, line_dash='dash', line_color='red', row=2, col=1)

fig.update_yaxes(title_text='Price', type='log', row=1, col=1)
fig.update_layout(title='Stock-to-Flow Ratio',
                  legend=dict(orientation="h"),
                  width=1400, height=700)

fig.show()

In [ ]:
# PlanBTC RSI strategy
import bt
from technical_indicator_utils import get_rsi
from strategy_utils import get_planBTC_strategy
from backtest_utils import signal_strategy, buy_and_hold_strategy, signal_above_strategy

summaries = {'OpenPrice': 'first', 'HighPrice': 'max', 'LowPrice': 'min', 'ClosePrice': 'last'}

df_planBTC = df_price_ohlc.resample('1M').agg(summaries)
# remove incomplete candle
df_planBTC = df_planBTC[:-1]

df_planBTC['RSI'] = get_rsi(df_planBTC['ClosePrice'])
df_planBTC = df_planBTC['2011-04-01':]
df_planBTC.dropna(inplace=True)

df_planBTC['Signal'] = get_planBTC_strategy(df_planBTC['RSI'].copy())

# backtesting
initial_value = df_planBTC['ClosePrice'][0]
bt_plan_BTC = signal_strategy(df_planBTC[['ClosePrice']].copy(), df_planBTC[['Signal']].copy(), 'planBTC', initial_value)
bt_buy_and_hold = buy_and_hold_strategy(df_planBTC[['ClosePrice']].copy(), name='buy_and_hold', _initial_capital=initial_value)

bt_results = bt.run(bt_buy_and_hold, bt_plan_BTC)

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    specs=[[{'secondary_y': True}],[{}]],
    vertical_spacing=0.01, 
    row_heights=[0.7, 0.3])

# Add traces
# Price
fig.add_trace(
    go.Scatter(x=df_planBTC.index, y=df_planBTC['ClosePrice'], name='Price', legendgroup = '1'),
        secondary_y=True)
# RSI
fig.add_trace(
    go.Scatter(x=df_planBTC.index, y=df_planBTC['RSI'], name='RSI', mode='markers+lines', line_color='purple', legendgroup = '1'),
        secondary_y=False)
# RSI range 50-90
fig.add_hrect(y0=50, y1=90, line_width=0, fillcolor='purple', opacity=0.2, secondary_y=False)
# Buy signal
fig.add_trace(go.Scatter(x=df_planBTC[df_planBTC['Signal'] == 1.0].index, 
                         y=df_planBTC[df_planBTC['Signal'] == 1.0]['ClosePrice'],
                         name='Buy',
                         legendgroup = '1',
                         mode='markers',
                         marker=dict(
                            size=10, symbol='triangle-up', color='green')),
        secondary_y=True)
# Sell Signal
fig.add_trace(go.Scatter(x=df_planBTC[df_planBTC['Signal'] == -1.0].index, 
                         y=df_planBTC[df_planBTC['Signal'] == -1.0]['ClosePrice'],
                         name='Sell',
                         legendgroup = '1',
                         mode='markers',
                         marker=dict(
                            size=10, symbol='triangle-down', color='red')),
        secondary_y=True)

# Results
fig.add_trace(go.Scatter(x=df_planBTC.index,
                         y=bt_results._get_series(None).rebase()['planBTC'],
                         #y=bt_results.prices['planBTC'],
                         line=dict(color='red', width=2),
                         name='PlanBTC',
                         legendgroup = '2'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_planBTC.index,
                         y=bt_results._get_series(None).rebase()['buy_and_hold'],
                         #y=bt_results.prices['buy_and_hold'],
                         line=dict(color='gray', width=2),
                         name='Buy and Hold',
                         legendgroup = '2'), row=2, col=1)

# Add figure title
fig.update_layout(
    title_text='PlanB@100TrillionUSD PlanBTC.com - Monthly Closing Data',
    width=1200, height=800,
    legend_tracegroupgap = 360,
    hovermode='x unified')

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=True, row=1, col=1)
fig.update_yaxes(title_text='RSI', secondary_y=False, row=1, col=1)
fig.update_yaxes(title_text='Trading Results', type='log', row=2, col=1)

fig.show()

In [ ]:
bt_results.display()

In [ ]:
bt_results.get_transactions(strategy_name='planBTC')

In [ ]:
#https://coinmetrics.io/introducing-the-revamped-coin-metrics-python-api-client/
from coinmetrics.api_client import CoinMetricsClient

gather_new_data = False
filename = 'data/on-chain-indicators.parquet'
columns = ['PriceUSD',
    'SplyCur', #The sum of all native units ever created at the end of that interval.
    'AdrActCnt', #Addresses, active, count
    'AdrBalCnt', #Addresses, any balance, count
    'AdrBalNtv0.01Cnt', #Addresses, with balance, greater than 0.01 native units, count
    'AdrBalNtv0.1Cnt', #Addresses, with balance, greater than 0.1 native units, count
    'AdrBalNtv1Cnt', #Addresses, with balance, greater than 1 native units, count
    'AdrBalNtv10Cnt', #Addresses, with balance, greater than 10 native units, count
    'AdrBalNtv100Cnt', #Addresses, with balance, greater than 100 native units, count
    'AdrBalNtv1KCnt', #Addresses, with balance, greater than 1000 native units, count
    'AdrBalNtv10KCnt', #Addresses, with balance, greater than 10000 native units, count
    'CapMrktCurUSD', #Capitalization, market, current supply, USD
    'CapMVRVCur', #Capitalization, MVRV, current supply
    'CapRealUSD', #Capitalization, realized, USD
    'NVTAdj', #NVT, adjusted
    'NVTAdj90', #NVT, adjusted, 90d MA
    'HashRate', #Hash rate, mean
    'TxTfrValAdjUSD' #Transactions, transfers, value, adjusted, USD
]

metrics = pd.DataFrame(columns=columns)



try:
    table = pq.read_table(filename)
    metrics = table.to_pandas()
    last_date = metrics.index[-1].date()
    gather_new_data = last_date < date.today() - relativedelta(days = 1)
except FileNotFoundError:
    last_date = datetime.strptime('2009-01-01', '%Y-%m-%d').date()
    gather_new_data = True

end_date = last_date + relativedelta(days = 80)
if (end_date >= date.today()):
    end_date = date.today() - relativedelta(days = 1)

if (gather_new_data):
    # Obtain metrics by specifying the asset, date interval and frequency.
    # All metrics available: https://docs.coinmetrics.io/info/metrics
    num_request = 1
    while True:
        client = CoinMetricsClient()
        coinmetrics = client.get_asset_metrics(
            assets='btc',
            metrics=columns,
            page_size=10000,
            start_time=last_date,
            end_time=end_date,
            frequency='1d'
            )
        coinmetrics = pd.DataFrame(coinmetrics)
        coinmetrics['time'] = pd.to_datetime(coinmetrics['time'])
        coinmetrics = coinmetrics.set_index('time')
        coinmetrics['PriceUSD'] = pd.to_numeric(coinmetrics['PriceUSD'])
        coinmetrics['SplyCur'] = pd.to_numeric(coinmetrics['SplyCur'])
        coinmetrics['AdrActCnt'] = pd.to_numeric(coinmetrics['AdrActCnt'])
        coinmetrics['AdrBalCnt'] = pd.to_numeric(coinmetrics['AdrBalCnt'])
        coinmetrics['AdrBalNtv0.01Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv0.01Cnt'])
        coinmetrics['AdrBalNtv0.1Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv0.1Cnt'])
        coinmetrics['AdrBalNtv1Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv1Cnt'])
        coinmetrics['AdrBalNtv10Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv10Cnt'])
        coinmetrics['AdrBalNtv100Cnt'] = pd.to_numeric(coinmetrics['AdrBalNtv100Cnt'])
        coinmetrics['AdrBalNtv1KCnt'] = pd.to_numeric(coinmetrics['AdrBalNtv1KCnt'])
        coinmetrics['AdrBalNtv10KCnt'] = pd.to_numeric(coinmetrics['AdrBalNtv10KCnt'])
        coinmetrics['CapMrktCurUSD'] = pd.to_numeric(coinmetrics['CapMrktCurUSD'])
        coinmetrics['CapRealUSD'] = pd.to_numeric(coinmetrics['CapRealUSD'])
        coinmetrics['CapMVRVCur'] = pd.to_numeric(coinmetrics['CapMVRVCur'])
        coinmetrics['NVTAdj'] = pd.to_numeric(coinmetrics['NVTAdj'])
        coinmetrics['NVTAdj90'] = pd.to_numeric(coinmetrics['NVTAdj90'])
        coinmetrics['HashRate'] = pd.to_numeric(coinmetrics['HashRate'])
        coinmetrics['TxTfrValAdjUSD'] = pd.to_numeric(coinmetrics['TxTfrValAdjUSD'])

        metrics = pd.concat([metrics, coinmetrics])
        metrics.drop_duplicates(inplace=True)

        if (end_date >= date.today()):
            break

        end_date = end_date + relativedelta(days = 80)
        
        num_request += 1
        if (num_request > 10):
            num_request = 1
            sleep(10)

table = pa.Table.from_pandas(metrics)
pq.write_table(table, filename)


In [ ]:
metrics

In [ ]:
metrics

In [ ]:
#NVT
nvt_metrics = metrics['2012-11-28':]
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=nvt_metrics.index, y=nvt_metrics['PriceUSD'], name='Price'),
        secondary_y=True)

fig.add_trace(
    go.Scatter(x=nvt_metrics.index, y=nvt_metrics['NVTAdj90'], name='NVT'),
        secondary_y=False)

fig.add_hrect(y0=40, y1=90, line_width=0, fillcolor='purple', opacity=0.2, secondary_y=False)

# Add figure title
fig.update_layout(
    title_text='BTC Price vs NVT',
    hovermode='x unified',
    legend=dict(orientation='h'))

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='NVT', secondary_y=False)

fig.show()

In [ ]:
#MVRV
import plotly.graph_objects as go
from plotly.subplots import make_subplots

mvrv_data = metrics[['PriceUSD', 'CapMVRVCur', 'CapMrktCurUSD', 'CapRealUSD']].copy()
mvrv_data['Z_Score'] = (mvrv_data['CapMrktCurUSD'] - mvrv_data['CapRealUSD']) / mvrv_data['CapMrktCurUSD'].expanding().std(ddof=0)
mvrv_data = mvrv_data['2012-11-28':]
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=mvrv_data.index, y=mvrv_data['PriceUSD'], name='Price'),
        secondary_y=True)
fig.add_trace(
    go.Scatter(x=mvrv_data.index, y=mvrv_data['CapMVRVCur'], name='MVRV'),
        secondary_y=False)
fig.add_trace(
    go.Scatter(x=mvrv_data.index, y=mvrv_data['Z_Score'], name='Z-Score'),
        secondary_y=False)
fig.add_hrect(y0=7, y1=9, line_width=0, fillcolor='red', opacity=0.2, secondary_y=False)
fig.add_hrect(y0=-0.4, y1=0.1, line_width=0, fillcolor='green', opacity=0.2, secondary_y=False)

# Add figure title
fig.update_layout(
    title_text='MVRV Ratio Z-Score',
    legend=dict(orientation="h"),
    width=1400, height=700,
    hovermode='x unified')

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='MVRV', secondary_y=False)

fig.show()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name='Price'),
        secondary_y=False)

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['AdrBalCnt'], name='Number of Addresses'),
        secondary_y=True)

# Add figure title
fig.update_layout(
    title_text='Number of Addresses with a Non-Zero Balance',
    legend=dict(orientation="h"))

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=False)
fig.update_yaxes(title_text='Addresses with a Non-Zero Balance', secondary_y=True)

fig.show()

In [ ]:
# Bitcoin Realized Cap
# Colorscales: https://plotly.com/python/builtin-colorscales/
data = metrics['2011-01-01':].copy()
data['Distance'] = (np.log(data['CapMrktCurUSD']) - np.log(data['CapRealUSD']))

fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, 
                         y=data['CapMrktCurUSD'],
                         name='Market Cap',
                         mode='markers',
                         marker=dict(
                            size=6,
                            color=data['Distance'], # set color equal to a variable
                            colorscale='Rainbow',
                            colorbar=dict(
                                title='Distance between Market and Realized Cap',
                                titleside='right'
                            ),
                            showscale=True)
                        ))
fig.add_trace(go.Scatter(x=data.index, 
                         y=data['CapRealUSD'],
                         name='Realized Cap',
                         mode='lines'))

fig.update_yaxes(type="log")

fig.update_layout(title='Bitcoin Realized Cap',
                  legend=dict(orientation='h'),
                  width=1400, height=600)

fig.show()

In [ ]:
#NUPL
import plotly.graph_objects as go
from plotly.subplots import make_subplots

nupl_data = metrics['2012-11-28':][['PriceUSD', 'CapMrktCurUSD', 'CapRealUSD']]
nupl_data['nupl'] = (nupl_data['CapMrktCurUSD'] - nupl_data['CapRealUSD']) / nupl_data['CapMrktCurUSD']
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=nupl_data.index, y=nupl_data['PriceUSD'], name='Price'),
        secondary_y=True)
fig.add_trace(
    go.Scatter(x=nupl_data.index, y=nupl_data['nupl'], name='NUPL'),
        secondary_y=False)

#Capitulation
fig.add_hrect(y0=-1.0, y1=0.0,
    line_width=0, fillcolor='blue',
    opacity=0.3,
    annotation_text="Capitulation", annotation_position="top left",
    secondary_y=False)

#Hope/Fear
fig.add_hrect(y0=0.0, y1=0.25,
    line_width=0,
    fillcolor='green',
    opacity=0.3,
    annotation_text="Hope / Fear", annotation_position="top left",
    secondary_y=False)

#Optimism/Anxiety
fig.add_hrect(y0=0.25, y1=0.50,
    line_width=0,
    fillcolor='yellow',
    opacity=0.3,
    annotation_text="Optimism / Anxiety", annotation_position="top left",
    secondary_y=False)

#Belief/Denial
fig.add_hrect(y0=0.50, y1=0.75,
    line_width=0,
    fillcolor='orange',
    opacity=0.3,
    annotation_text="Belief / Denial",
    annotation_position="top left",
    secondary_y=False)

#Euphoria/Greed
fig.add_hrect(y0=0.75, y1=1.0,
    line_width=0,
    fillcolor='red',
    opacity=0.3,
    annotation_text="Euphoria / Greed", annotation_position="top left",
    secondary_y=False)

# Add figure title
fig.update_layout(
    title_text='Net Unrealized Profit/Loss (NUPL)',
    legend=dict(orientation="h"),
    width=1400, height=800,
    hovermode='x unified')

# Set y-axes titles
fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='NUPL', secondary_y=False)

fig.show()

In [ ]:
# Bitcoin Price - NUPL
time_period = '1W'
nupl_data = metrics['2012-11-28':][['PriceUSD', 'CapMrktCurUSD', 'CapRealUSD']].resample(time_period).last()
nupl_data['nupl'] = (nupl_data['CapMrktCurUSD'] - nupl_data['CapRealUSD']) / nupl_data['CapMrktCurUSD']

nupl_data['nupl_range'] = 0 # Capitulation
nupl_data['nupl_range'] = np.where((nupl_data['nupl'] > 0.0) & (nupl_data['nupl'] <= 0.25), 1, nupl_data['nupl_range']) # Hope/Fear
nupl_data['nupl_range'] = np.where((nupl_data['nupl'] > 0.25) & (nupl_data['nupl'] <= 0.50), 2, nupl_data['nupl_range']) # Optimism/Anxiety
nupl_data['nupl_range'] = np.where((nupl_data['nupl'] > 0.50) & (nupl_data['nupl'] <= 0.75), 3, nupl_data['nupl_range']) # Belief/Denial
nupl_data['nupl_range'] = np.where(nupl_data['nupl'] > 0.75, 4, nupl_data['nupl_range']) # Euphoria/Greed

fig = go.Figure()
fig.add_trace(go.Scatter(x=nupl_data.index,
                         y=nupl_data['PriceUSD'],
                         name='Price',
                         mode='markers',
                         marker=dict(
                            size=8,
                            color=nupl_data['nupl_range'], # set color equal to a variable
                            colorscale='jet',
                            colorbar=dict(
                                title="NUPL",
                                titleside="right",
                                tickmode="array",
                                tickvals=[0, 1, 2, 3, 4],
                                ticktext=["Capitulation", "Hope/Fear", "Optimism/Anxiety", "Belief/Denial", "Euphoria/Greed"],
                                ticks="outside"
                            )
                        )
                    )
)

fig.update_yaxes(type="log")

fig.update_layout(title='Bitcoin Price / NUPL',
                  width=1200, height=600)

fig.show()

## Bitcoin Supply Distribution

We divide network entities according to their Bitcoin holdings into the following marine species:

* Shrimps (<1 BTC)
* Crab (1-10 BTC)
* Octopus (10-50 BTC)
* Fish (50-100 BTC)
* Dolphin (100-500 BTC)
* Shark (500-1,000 BTC)
* Whale (1,000-5,000 BTC)
* Humpback (>5,000 BTC).

In [ ]:
metrics['shrimps'] = metrics['AdrBalCnt'] - metrics['AdrBalNtv1Cnt']
metrics['crab'] = metrics['AdrBalNtv1Cnt'] - metrics['AdrBalNtv10Cnt']
metrics['octopus-fish'] = metrics['AdrBalNtv10Cnt'] - metrics['AdrBalNtv100Cnt']
metrics['dolphin-shark'] = metrics['AdrBalNtv100Cnt'] - metrics['AdrBalNtv1KCnt']
metrics['whale-humpback'] = metrics['AdrBalNtv1KCnt']

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['shrimps'], name='Shrimp', mode='lines'),
    secondary_y=False)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['crab'], name='Crab', mode='lines'),
    secondary_y=False)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['octopus-fish'], name='Octopus-Fish', mode='lines'),
    secondary_y=False)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['dolphin-shark'], name='Dolphin-Shark', mode='lines'),
    secondary_y=False)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['whale-humpback'], name='Whale-Humpback', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='Addresses Count', type='log', secondary_y=False)

fig.update_layout(title='Supply distribution - Log scale',
                  width=1400, height=600)

fig.show()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['shrimps'], name='Shrimp', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='Addresses Count', secondary_y=False)

fig.update_layout(title='Supply distribution - Shrimp',
                  width=1400, height=600)

fig.show()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['crab'], name='Crab', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='Addresses Count', secondary_y=False)

fig.update_layout(title='Supply distribution - Crab',
                  width=1400, height=600)

fig.show()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['octopus-fish'], name='Octopus-Fish', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='Addresses Count', secondary_y=False)

fig.update_layout(title='Supply distribution - Octopus-Fish',
                  width=1400, height=600)

fig.show()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['dolphin-shark'], name='Dolphin-Shark', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='Addresses Count', secondary_y=False)

fig.update_layout(title='Supply distribution - Dolphin-Shark',
                  width=1400, height=600)

fig.show()

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['PriceUSD'], name="Price"),
    secondary_y=True)
fig.add_trace(
    go.Scatter(x=metrics.index, y=metrics['whale-humpback'], name='Whale-Humpback', mode='lines'),
    secondary_y=False)

fig.update_yaxes(title_text='Price', type='log', secondary_y=True)
fig.update_yaxes(title_text='Addresses Count', secondary_y=False)

fig.update_layout(title='Supply distribution - Whale-Humpback',
                  width=1400, height=600)

fig.show()

In [ ]:
import yfinance as yf
import math

# interval '1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo'
spy_ohlc_df = yf.download('SPY', start='2009-01-01', interval='1d')
spy_ohlc_df = spy_ohlc_df.dropna()

In [ ]:
from technical_indicator_utils import normalize

data_spy = spy_ohlc_df['2015-01-01':].copy()
data_btc = metrics['2015-01-01':].copy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_spy.index, y=normalize(np.log(data_spy['Adj Close'])),
                    mode='lines',
                    name='S&P500'))
fig.add_trace(go.Scatter(x=data_btc.index, y=normalize(np.log(data_btc['PriceUSD'])),
                    mode='lines',
                    name='BTC'))

fig.show()